I used those commands to connect to my google drive in order to save and retrieve the preprocessed data without starting from scratch


In [1]:
from google.colab import drive
drive.mount('/gdrive')

Mounted at /gdrive


In [2]:
%cd ../gdrive/MyDrive/bigdata


/gdrive/MyDrive/bigdata


In [ ]:
!ls


amazon-fine-food-reviews.zip  hashes.txt   Reviews.csv	X.pkl
database.sqlite		      kaggle.json  reviews.txt


In [3]:
!pip install pickle-mixin --user

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pickle-mixin: filename=pickle_mixin-1.0.2-py3-none-any.whl size=6008 sha256=845e1117f4b184640b3c8cdcc98a8067a4604770d0949f415b362d01dd95ea8f
  Stored in directory: /root/.cache/pip/wheels/d0/70/0b/673e09a7ed429660d22352a1b117b4f616a8fc054bdd7eb157
Successfully built pickle-mixin


In [5]:
import pandas as pd
import nltk
import re
import string
import random 
import numpy as np
import pickle
import scipy

used this to insert my kaggle key to download the dataset from kaggle without uploading it

In [ ]:
from google.colab import files
uploaded = files.upload()

In [ ]:
import os
os.environ['KAGGLE_CONFIG_DIR'] = "/content"

In [ ]:
!chmod 600 /content/kaggle.json

In [ ]:
!kaggle datasets download -d snap/amazon-fine-food-reviews

In [ ]:
from zipfile import ZipFile

In [ ]:
# Create a ZipFile Object and load sample.zip in it
with ZipFile('amazon-fine-food-reviews.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

i used the preprocessed procedure linked in the classroom. I basically remove stop words and i transform everything into tfidf vectors.

In [ ]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

from nltk.tokenize import word_tokenize #Used to extract words from documents
from nltk.stem import WordNetLemmatizer #Used to lemmatize words
from sklearn.feature_extraction.text import TfidfVectorizer

read dataset and shuffle it so that it is easier to use just a subset of it to do testings.

In [ ]:
#read data from csv and get the reviews
df = pd.read_csv('Reviews.csv')
reviews=df["Text"].tolist()
#shuffle to later divide in test and training without bias
random.shuffle(reviews)

remove contraddictions and abbreviations to simplify the analysis of documents

In [ ]:
#preprocess the reviews
contractions_dict = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "i'd": "I would",
  "i'd've": "I would have",
  "i'll": "I will",
  "i'll've": "I will have",
  "i'm": "I am",
  "i've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you will",
  "you'll've": "you will have",
  "you're": "you are",
  "you've": "you have"
}

# Regular expression for finding contractions
def multiple_replace(dict, text):
  # Create a regular expression  from the dictionary keys
  regex = re.compile("(%s)" % "|".join(map(re.escape, dict.keys())))

  # For each match, look-up corresponding value in dictionary
  return regex.sub(lambda mo: dict[mo.string[mo.start():mo.end()]], text)

reviews = [multiple_replace(contractions_dict, doc) for doc in reviews]
table = str.maketrans('', '', string.punctuation)
reviews = [doc.translate(table) for doc in reviews]
reviews = [re.sub(r'\d+', " ", doc) for doc in reviews]

lemmatize the document in order to group words that mean the same thing and so on to have an analysis on the semantic

In [ ]:
def get_lemmatized(doc):
  word_list = word_tokenize(doc)
  lemmatized_doc = ""
  for word in word_list:
    lemmatized_doc = lemmatized_doc + " " + lemmatizer.lemmatize(word)
  return lemmatized_doc

lemmatizer = WordNetLemmatizer()

reviews = [get_lemmatized(doc) for doc in reviews]

remove stopwords that don't capture things that are useful

In [ ]:
stopwords = nltk.corpus.stopwords.words('english') # Returns a list
stopwords = set(stopwords) # We want a set, because this is implemented with a hash table
                           # Checking the if condition in rem_stop costs O(1) in this way

def rem_stop(doc):
   word_list = word_tokenize(doc)
   cleaned_doc = ""
   for word in word_list:
     if word not in stopwords:
       cleaned_doc += " " + word
   return cleaned_doc


reviews = [rem_stop(doc) for doc in reviews]

vectorize the documents with tfidf to capture informations based on the frequencies of the words in the document.Not frequently used words are more useful.

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english', min_df=10) ## Corpus is in English
X = vectorizer.fit_transform(reviews)

used this to save reviews without redoing everything

In [ ]:
# #write reviews to file
with open('reviews.txt', 'w',encoding='utf8') as f:
    for item in reviews:
        f.write(item+",")

save the processed document on the drive to reload it without redo everything everytime i disconnect the environment

In [ ]:
import pickle
#write x to file
with open('X.pkl', 'wb') as f:
	pickle.dump(X, f)

# read obj from file
with open('X.pkl', 'rb') as f:
	obj = pickle.load(f)

this function reads the preprocessed documents saved previously 

In [21]:
import pickle

def readReviewsX():
  #read reviews from file
  with open('reviews.txt', 'r',encoding='utf8') as f:
      reviews = f.read().split(",")
      reviews = reviews[:-1]
  #read X from file
  with open('X.pkl', 'rb') as f:
      X = pickle.load(f)
  return reviews,X
rows,cols=X.shape

take just a percentage of the dataset to test everything on a small subset

In [18]:
def percentageOfDataset(reviews,X,percentage=1):
  reviewsOut=reviews[:int(len(reviews)*percentage)]
  XOut=X[:int(X.shape[0]*percentage)]
  return reviewsOut,XOut


(113690, 33897)


(22738, 33897)

this class contains some operations that are useful both for the normal lsh and for the lsh vectorized

In [ ]:
class Operations: 
    #generate a random vector direction used to calculate the hashes saved in the matrix m
    def generateU():
        u=np.random.normal(0,1,cols)
        u=scipy.sparse.csr_matrix(u)
        return u
    #sparse dot product for sparse sci py matrices
    def sparseProduct(x,y):
        return x.dot(y.transpose()).toarray()[0][0]
    #given u and a vector it returns the hash to be saved in the matrix m
    def h(x,u):
        return np.sign(Operations.sparseProduct(x,u))
    #returns the exact cosine similarity of 2 vectors
    def cosineSimilarity(x,y):
        r=Operations.sparseProduct(x,y)/(np.linalg.norm(x.toarray())*np.linalg.norm(y.toarray()))
        return r
    #returns the exact jaccard similarity of 2 vectors
    def jaccardSimilarity(x,y):
        intersection=len(x.intersection(y))
        union=len(x.union(y))
        return intersection/union

this is the hash class used to generate an hash function to be called on an input.
generate nBytes vectors, multiply it for the input and the resulting vector will have 1 if the elements are positive or 0 if <= 0.
This hash is used in the normal lsh not in the vectorized lsh that i wrote

In [ ]:
class Hash:
    def __init__(self,nBytes=50,inputDimension=10):
        self.nBytes=nBytes
        self.inputDimension=inputDimension
        self.reset()

    def reset(self):
        self.matrix=np.random.randn(self.nBytes,self.inputDimension)

    def generateHash(self,input):
        if type(input)==list:
            input=np.array(input)
        bools = (np.dot(input, self.matrix.T) > 0).astype('int')
        return ''.join(bools.astype('str'))



In [9]:
h=Hash()
h.generateHash(np.ones(10))

NameError: ignored

this class is responsible to perform the generation of the m matrix both for the train and for the test datasets , perform the lsh and compare the results for the approximated results and the exact results

In [11]:

class LshNormal:
    def __init__(self,X,reviews,b=10,r=10,testPercentage=0.005,nHashes=100):
        self.X=X
        self.reviews=reviews
        self.rows=X.shape[0]
        self.cols=X.shape[1]
        self.b=b
        self.r=r
        self.testPercentage=testPercentage
        self.nHashes=nHashes
        self.splitDataset()

    def setBR(self,b,r):
        self.b=b
        self.r=r
        
    def splitDataset(self):
        indexSplit=int(self.rows*self.testPercentage)
        self.testReviews,self.trainReviews=self.reviews[:indexSplit],self.reviews[indexSplit:]
        self.testX,self.trainX=self.X[:indexSplit],self.X[indexSplit:]
        self.rowsTest,self.colsTest=self.testX.shape
        self.rowsTrain,self.colsTrain=self.trainX.shape

    def generateM(self):
        self.mTest=[[]for i in range(self.rowsTest)]
        self.mTrain=[[]for i in range(self.rowsTrain)]
        for i in range(self.nHashes):
            u=Operations.generateU()
            for j in range(self.rowsTest):
                h=Operations.h(self.testX[j],u)
                self.mTest[j].append(h)
            for j in range(self.rowsTrain):
                h=Operations.h(self.trainX[j],u)
                self.mTrain[j].append(h)

    def lsh(self):
        self.dicts=[{} for i in range(self.b)]
        self.hashesUsed=[Hash(inputDimension=self.r) for i in range(self.b)]
        for i in range(self.b):
            for j in range(self.rowsTrain):
                v=self.mTrain[j][i*self.r:(i+1)*self.r]
                key=self.hashesUsed[i].generateHash(v)
                if key not in self.dicts[i]:
                    self.dicts[i][key]=set()
                self.dicts[i][key].add(j)

    def findExactCosineSimilars(self,threshold=0.5):
        self.exactSimilars={}
        for i,v in enumerate(self.testX):
            for j,u in enumerate(self.trainX):
                r=Operations.cosineSimilarity(v,u)
                if r>=threshold:
                    if i not in self.exactSimilars:
                        self.exactSimilars[i]=set()
                    self.exactSimilars[i].add(j)
        return self.exactSimilars

    def findCosineSimilarsApproximated(self):
        self.approximatedSimilars={}
        for i,v in enumerate(self.mTest):
            for j,hash in enumerate(self.hashesUsed):
                key=hash.generateHash(v[j*self.r:(j+1)*self.r])
                if key in self.dicts[j]:
                    for k in self.dicts[j][key]:
                        if i not in self.approximatedSimilars:
                            self.approximatedSimilars[i]=set()
                        self.approximatedSimilars[i].add(k)
        return self.approximatedSimilars
    
    def jaccardAB(self):
        self.A=set()
        self.B=set()
        for i in self.exactSimilars:
            for j in self.exactSimilars[i]:
                self.A.add((i,j))
        for i in self.approximatedSimilars:
            for j in self.approximatedSimilars[i]:
                self.B.add((i,j))
        return Operations.jaccardSimilarity(self.A,self.B)

    def optimizeRB(self,cosineSimilarity=0.2,nHashes=10):
      self.findExactCosineSimilars(cosineSimilarity)
      self.nHashes=nHashes
      maxTaken=0
      lsh.generateM()
      #automatic search for b and r
      maxJaccard=0
      maxR=0
      maxB=0
      for i in range(1,int(lsh.nHashes**0.5)):
        lsh.r=i
        lsh.b=int(lsh.nHashes/i)
        lsh.lsh()
        b=lsh.findCosineSimilarsApproximated()
        jacc=lsh.jaccardAB()
        if jacc>maxJaccard:
          maxJaccard=jacc
          maxR=lsh.r
          maxB=lsh.b
          maxTaken=Operations.jaccardSimilarity(self.A.intersection(self.B),self.A)
        lsh.b=i
        lsh.r=int(lsh.nHashes/i)
        lsh.lsh()
        b=lsh.findCosineSimilarsApproximated()
        jacc=lsh.jaccardAB()
        if jacc>maxJaccard:
          maxJaccard=jacc
          maxR=lsh.r
          maxB=lsh.b
          maxTaken=Operations.jaccardSimilarity(self.A.intersection(self.B),self.A)

      print([maxJaccard,maxR,maxB])
      print(maxTaken)
      self.r=maxR
      self.b=maxB
      return [maxJaccard,maxR,maxB]


  



this is the vectorized lsh class.
it generates the matrix m using vectorized operations with numpy and performs the lsh and exact similars functions in a vectorized ways.
This significately speed things up because operations are parallelized and optimized in c to speed things up.

In [10]:

class LshVectorized:

    def __init__(self,X,reviews,b=10,r=10,testPercentage=0.005,nHashes=100):
        self.X=X
        self.reviews=reviews
        self.rows=X.shape[0]
        self.cols=X.shape[1]
        self.b=b
        self.r=r
        self.testPercentage=testPercentage
        self.nHashes=nHashes
        self.splitDataset()

    def setBR(self,b,r):
        self.b=b
        self.r=r
        
    def splitDataset(self):
        indexSplit=int(self.rows*self.testPercentage)
        self.testReviews,self.trainReviews=self.reviews[:indexSplit],self.reviews[indexSplit:]
        self.testX,self.trainX=self.X[:indexSplit],self.X[indexSplit:]
        self.rowsTest,self.colsTest=self.testX.shape
        self.rowsTrain,self.colsTrain=self.trainX.shape

    def generateMMM(self):
      allU=np.random.normal(0,1,[self.cols,self.nHashes])
      allU=scipy.sparse.csr_matrix(allU)
      array=np.array((self.testX * allU).toarray())
      self.mTest=np.sign(array)
      array=np.array((self.trainX * allU).toarray())
      self.mTrain=np.sign(array)

    def lshMM(self):
      hashes=np.random.randn(self.b,30,self.r)
      self.hashesMM=hashes
      signatureTrain=self.mTrain[:,0:self.r*self.b].reshape(-1,self.b,self.r)
      self.signatureTrain=np.einsum("abr, bdr-> ab",signatureTrain,hashes)
      signatureTest=self.mTest[:,0:self.r*self.b].reshape(-1,self.b,self.r)
      self.signatureTest=np.einsum("abr, bdr-> ab",signatureTest,hashes)
 
    def findCosineSimilarsApproximatedMM(self):
        self.approximatedSimilars={}
        for i,signature in enumerate(self.signatureTest):
          self.approximatedSimilars[i]=set((self.signatureTrain==signature).nonzero()[0])
        return self.approximatedSimilars
    
    def findCosineSimilarsApproximatedMMSingular(self,i):
        return set((self.signatureTrain==self.signatureTest[i,:]).nonzero()[0])

    def findExactCosineSimilarsMM(self,threshold=0.5):
        self.exactSimilars={}
        sim=self.testX.dot(self.trainX.T)
        normTest=scipy.sparse.linalg.norm(self.testX,axis=1)
        normTrain=scipy.sparse.linalg.norm(self.trainX,axis=1)
        sim=sim/normTrain
        sim=(sim.T/normTest).T
        similars=(sim>=threshold).nonzero()
        for i in range(len(similars[0])):
          if similars[0][i] not in self.exactSimilars:
            self.exactSimilars[similars[0][i]]=set()
          self.exactSimilars[similars[0][i]].add(similars[1][i])
        return self.exactSimilars

    def optimizeRBMM(self,cosineSimilarity=0.2,nHashes=10):
      self.findExactCosineSimilarsMM(cosineSimilarity)
      self.nHashes=nHashes
      maxTaken=0
      lsh.generateMMM()
      #automatic search for b and r
      maxJaccard=0
      maxR=0
      maxB=0
      for i in range(1,int(lsh.nHashes**0.5)):
        lsh.r=i
        lsh.b=int(lsh.nHashes/i)
        lsh.lshMM()
        b=lsh.findCosineSimilarsApproximatedMM()
        jacc=lsh.jaccardAB()
        if jacc>maxJaccard:
          maxJaccard=jacc
          maxR=lsh.r
          maxB=lsh.b
          maxTaken=Operations.jaccardSimilarity(self.A.intersection(self.B),self.A)
        lsh.b=i
        lsh.r=int(lsh.nHashes/i)
        lsh.lshMM()
        b=lsh.findCosineSimilarsApproximatedMM()
        jacc=lsh.jaccardAB()
        if jacc>maxJaccard:
          maxJaccard=jacc
          maxR=lsh.r
          maxB=lsh.b
          maxTaken=Operations.jaccardSimilarity(self.A.intersection(self.B),self.A)
      print([maxJaccard,maxR,maxB])
      print(maxTaken)
      self.r=maxR
      self.b=maxB
      return [maxJaccard,maxR,maxB]
    
    def jaccardAB(self):
        self.A=set()
        self.B=set()
        for i in self.exactSimilars:
            for j in self.exactSimilars[i]:
                self.A.add((i,j))
        for i in self.approximatedSimilars:
            for j in self.approximatedSimilars[i]:
                self.B.add((i,j))
        return Operations.jaccardSimilarity(self.A,self.B)


In [19]:
reviews,X=readReviewsX()
reviews,X=percentageOfDataset(reviews,X,0.2)
X.shape


(4547, 33897)

In [13]:
lsh=LshNormal(X,reviews,nHashes=100,r=5,b=20)


In [ ]:
lsh.r=10
lsh.b=20
lsh.nHashes=lsh.r*lsh.b
lsh.generateMMM()
lsh.lshMM()

In [ ]:
lsh.findExactCosineSimilarsMM(0.301)

In [ ]:
lsh.exactSimilars.keys()

dict_keys([18, 22, 34, 41, 43, 51, 54, 58, 59, 61, 69, 71, 74, 75, 76, 82, 83, 87, 92, 96, 100, 105, 109, 115, 119, 121, 122, 123, 125, 127, 128, 129, 131, 133, 147, 151, 152, 153, 154, 159, 162, 173, 174, 180, 191, 193, 197, 213, 214, 218, 221, 223, 227, 230, 237, 241, 248, 249, 251, 253, 254, 257, 263, 264, 265, 270, 277, 278, 282, 287, 289, 290, 291, 295, 298, 300, 306, 308, 309, 313, 315, 318, 320, 322, 327, 337, 341, 345, 347, 350, 353, 354, 356, 357, 362, 365, 367, 377, 380, 389, 393, 400, 401, 402, 404, 407, 410, 411, 413, 417, 418, 420, 421, 422, 423, 428, 429, 430, 435, 442, 444, 446, 448, 449, 451, 453, 456, 458, 461, 466, 468, 473, 477, 485, 487, 490, 492, 502, 503, 509, 511, 512, 513, 515, 517, 518, 521, 522, 524, 525, 528, 536, 542, 545, 547, 548, 559, 564, 566, 567, 569, 571, 573, 577, 579, 584, 586, 589, 590, 591, 593, 595, 598, 602, 608, 613, 619, 622, 624, 628, 629, 632, 633, 634, 649, 656, 657, 658, 664, 668, 670, 674, 676, 683, 685, 689, 690, 698, 701, 708, 722, 724,

In [ ]:
similars=lsh.findCosineSimilarsApproximatedMM()

In [ ]:
similar=lsh.findCosineSimilarsApproximatedMMSingular(0)

In [ ]:
lsh.jaccardAB()



0.0028244411846011725

In [ ]:
print(Operations.jaccardSimilarity(lsh.A.intersection(lsh.B),lsh.A))

0.15774959571027322


In [ ]:
print(f"dima={len(lsh.A)}, dimB={len(lsh.B)} , difference={len(lsh.B.difference(lsh.A))} , intersection={len(lsh.B.intersection(lsh.A))} ")

dima=58745, dimB=3231525 , difference=3222258 , intersection=9267 


In [ ]:
lsh.testX.shape

(28, 33897)

In [ ]:
fromsim=0.4
top=1-np.arccos(fromsim)/np.pi
top

0.6309898804344547

In [ ]:
fromp=0.6
tosim=np.cos((1-fromp)*np.pi)
tosim

0.30901699437494745

In [ ]:
a=lsh.findExactCosineSimilarsMM(0.4)
a

In [ ]:
a=lsh.findExactCosineSimilars(0.3)
a

In [ ]:
lsh.exactSimilars=a

In [ ]:
lsh.generateMMM()
print(lsh.mTest.shape,lsh.mTrain.shape)

(852, 100) (169684, 100)


In [ ]:
lsh.generateM()
lsh.mTest.shape
lsh.mTrain.shape

In [ ]:
mTest=lsh.mTest
mTrain=lsh.mTrain


In [ ]:
lsh.mTest=mTest
lsh.mTrain=mTrain
len(lsh.mTest[0])

NameError: ignored

In [ ]:
lsh.lsh()

In [ ]:
lsh.r=33
lsh.b=3
lsh.lshMM()


In [ ]:
lsh.findCosineSimilarsApproximatedMM()
"ciao"

'ciao'

In [ ]:
lsh.r=10
lsh.b=100

lsh.lsh()
b=lsh.findCosineSimilarsApproximated()


In [ ]:
#automatic search for b and r
maxJaccard=0
maxR=0
maxB=0

for i in range(1,int(lsh.nHashes**0.5)):

  lsh.r=i
  lsh.b=int(lsh.nHashes/i)
  lsh.lsh()
  b=lsh.findCosineSimilarsApproximated(0.123)
  jacc=lsh.jaccardAB()
  if jacc>maxJaccard:
    maxJaccard=jacc
    maxR=lsh.r
    maxB=lsh.b



  lsh.b=i
  lsh.r=int(lsh.nHashes/i)
  lsh.lsh()
  b=lsh.findCosineSimilarsApproximated(0.123)
  jacc=lsh.jaccardAB()
  if jacc>maxJaccard:
    maxJaccard=jacc
    maxR=lsh.r
    maxB=lsh.b

print([maxJaccard,maxR,maxB])

  

  


AttributeError: ignored

In [ ]:
lsh.optimizeRB(0.3,10)
lsh.optimizeRB(0.2,10)
lsh.optimizeRB(0.4,10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in double_scalars
  from ipykernel import kernelapp as app


In [ ]:
lsh.optimizeRB(0.2,30)
lsh.optimizeRB(0.3,30)
lsh.optimizeRB(0.4,30)
lsh.optimizeRB(0.2,40)
lsh.optimizeRB(0.3,40)
lsh.optimizeRB(0.4,40)
lsh.optimizeRB(0.2,50)
lsh.optimizeRB(0.3,50)
lsh.optimizeRB(0.4,50)



KeyboardInterrupt: ignored

In [ ]:
lsh.optimizeRBMM(0.2,1000)
lsh.optimizeRBMM(0.3,100)
lsh.optimizeRBMM(0.4,100)


[0.008841119213987123, 11, 9]
0.025572569612991484
[0.008108752683043167, 16, 6]
0.010611735330836454
[0.05005324813631523, 25, 4]
0.050106609808102345


[0.05005324813631523, 25, 4]

In [ ]:
lsh.optimizeRB(0.2,200)

lsh.optimizeRB(0.3,200)

lsh.optimizeRB(0.4,200)


[0.008708272859216255, 13, 15]
0.017191977077363897
[0.0020149683362118594, 10, 20]
0.1794871794871795


KeyboardInterrupt: ignored

In [ ]:
lsh.optimizeRB(0.2,1000)
lsh.optimizeRB(0.3,1000)
lsh.optimizeRB(0.4,1000)


In [ ]:
lsh.r=5
lsh.b=2
lsh.nHashes=lsh.b*lsh.r
lsh.generateMMM()
lsh.lshMM()
lsh.findCosineSimilarsApproximatedMM()

In [ ]:
print(lsh.jaccardAB())
print(Operations.jaccardSimilarity(lsh.a.intersection(lsh.b),lsh.a))
# intersection=lsh.a.intersection(lsh.b)
# print(len(intersection),len(lsh.a))


0.0003733264675592173


AttributeError: ignored